In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.signal import find_peaks, periodogram, welch
import numpy as np
from statsmodels.tsa.seasonal import STL
from sktime.transformations.series.outlier_detection import HampelFilter



df = pd.read_csv('./dataset/order_imported.csv')

assumptions = """Assumptions:

1. Order import -> Fraud Check -> Authorized Order -> Pick Ticket
2. No missing values exist
3. Cycles happening daily / weekly / business
4. Trends happen due to holidays on Black Friday and Christmas
5. 30 minute windows
6. All datasets are synced to same timezones and timestamps
"""

print(assumptions)

# Convert 'ds' to datetime and set 30-minute frequency
df['ds'] = pd.to_datetime(df['ds'])
df = df.set_index('ds').asfreq('30min').reset_index()

# Linearly interpolate missing rows
df = df.interpolate(method='linear')

In [ ]:
# 1. Preliminary information

def print_df_info(df : pd.DataFrame, name : str) -> None:
    print(f"Preliminary information for {name}")
    print(f"Size: {len(df)}\n")
    print(f"'y' min and max: {df['y'].min()}, {df['y'].max()}\n")
   
    print("column dtypes:")
    for col in df.columns:
        print(f"  {col}: {df[col].dtype}")


# Reverse the dataframe to have chronological order (oldest first)
df = df.iloc[::-1].reset_index(drop=True)

SIZE = len(df)
SPLIT = 1680
print_df_info(df, "Imported Orders")

In [ ]:
# 2. Data Summaries

def print_y_summaries(df : pd.DataFrame, name : str) -> None:
    """
    print global y-statistics
    
    Parameters:
    df: DataFrame with 'y' column
    """
    print(f"Global Stats: 'y' ({name}):\n")
    print(f"Mean: {df['y'].mean():.2f}")
    print(f"Median: {df['y'].median():.2f}")
    print(f"Std: {df['y'].std():.2f}")
    print(f"IQR: {df['y'].quantile(0.75) - df['y'].quantile(0.25):.2f}")
    print(f"Min: {df['y'].min()}")
    print(f"Max: {df['y'].max()}")
    print(f"Skewness: {df['y'].skew():.2f}")
    print(f"Kurtosis: {df['y'].kurtosis():.2f}")


def plot_rolling_y(df: pd.DataFrame, name : str, n: int) -> None:
    """
    Calculate and plot rolling mean, median, IQR, std for 'y' column
    
    Parameters:
    df: DataFrame with 'y' column
    n: rolling window size
    """
    # Calculate rolling statistics
    rolling_mean = df['y'].rolling(window=n).mean()
    rolling_median = df['y'].rolling(window=n).median()
    rolling_std = df['y'].rolling(window=n).std()
    rolling_iqr = df['y'].rolling(window=n).apply(lambda x: x.quantile(0.75) - x.quantile(0.25))
    
    # Create subplots
    fig, axes = plt.subplots(4, 1, figsize=(6, 5))
    fig.suptitle(f"Rolling Stats: {name}, 'y' (Window Size: {n})", fontsize=16)
    
    # Plot rolling mean
    axes[0].plot(df.index, rolling_mean, label=f'Rolling Mean (n={n})', color='blue')
    axes[0].set_ylabel('Mean')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot rolling median
    axes[1].plot(df.index, rolling_median, label=f'Rolling Median (n={n})', color='green')
    axes[1].set_ylabel('Median')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Plot rolling std
    axes[2].plot(df.index, rolling_std, label=f'Rolling Std (n={n})', color='red')
    axes[2].set_ylabel('Std')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    # Plot rolling IQR
    axes[3].plot(df.index, rolling_iqr, label=f'Rolling IQR (n={n})', color='orange')
    axes[3].set_ylabel('IQR')
    axes[3].set_xlabel('Time (30 mins)')
    axes[3].legend()
    axes[3].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print_y_summaries(df, "Imported Orders")
for n in (48, 168, 1460):
    plot_rolling_y(df, "Imported Orders", n=n)



In [ ]:
# 3. Vizualizations
def graph_y_time(df : pd.DataFrame, name : str):
    plt.figure(figsize=(10, 4))
    plt.plot(df.index, df['y'], label=f'{name} (y)')
    plt.xlabel('Timestamp')
    plt.ylabel('Imported Orders (y)')
    plt.title('Imported Orders Over Time')
    plt.legend()
    plt.show()


def plot_y_analysis(df: pd.DataFrame, name: str):
    """
    Comprehensive visualization suite for y-feature analysis
    """
            
    # Create a comprehensive subplot figure with all visualizations
    fig = plt.figure(figsize=(15, 12))
    gs = fig.add_gridspec(4, 3, hspace=0.5, wspace=0.3)
    fig.suptitle(f'{name}: Comprehensive Y-Feature Analysis', fontsize=16)
    
    # Zoomed windows (random + recent)
    ax1 = fig.add_subplot(gs[0, :2])
    random_start = len(df) // 2 - 100
    random_end = random_start + 200
    ax1.plot(df.index[random_start:random_end], df['y'].iloc[random_start:random_end])
    ax1.set_title('Random Window (Middle 200 points)')
    ax1.set_ylabel('y')
    ax1.grid(True, alpha=0.3)
    
    ax2 = fig.add_subplot(gs[0, 2])
    ax2.plot(df.index[-200:], df['y'].iloc[-200:], color='orange')
    ax2.set_title('Recent Window (Last 200 points)')
    ax2.set_ylabel('y')
    ax2.grid(True, alpha=0.3)
    
    # Rolling mean/std overlay
    ax3 = fig.add_subplot(gs[1, :])
    window = 48
    rolling_mean = df['y'].rolling(window=window).mean()
    rolling_std = df['y'].rolling(window=window).std()
    
    ax3.plot(df.index, df['y'], alpha=0.5, label='Raw Data', linewidth=0.8)
    ax3.plot(df.index, rolling_mean, label=f'Rolling Mean (n={window})', color='red', linewidth=2)
    ax3.fill_between(df.index, rolling_mean - rolling_std, rolling_mean + rolling_std, 
                     alpha=0.2, color='red', label='±1 Std Dev')
    ax3.set_ylabel('y')
    ax3.set_title('Rolling Mean/Std Overlay')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Histogram + KDE
    ax4 = fig.add_subplot(gs[2, :])
    ax4.hist(df['y'], bins=30, density=True, alpha=0.7, edgecolor='black', label='Histogram')
    df['y'].plot(kind='kde', ax=ax4, color='red', linewidth=2, label='KDE')
    ax4.set_xlabel('y')
    ax4.set_ylabel('Density')
    ax4.set_title('Distribution (Histogram + KDE)')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
        
    # Boxplot by time bucket (48/168/1460)
    buckets = [48, 168, 1460]
    for i, bucket_size in enumerate(buckets):
        ax = fig.add_subplot(gs[3, i])
        df_copy = df.copy()
        df_copy['bucket'] = df_copy.index // bucket_size
        df_copy.boxplot(column='y', by='bucket', ax=ax)
        ax.set_title(f'Bucket Size: {bucket_size}')
        ax.set_xlabel('Bucket')
        ax.set_ylabel('y')
        ax.get_figure().suptitle('')  # Remove automatic title from boxplot
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
        
        # 48 labels were messy
        if bucket_size == 48:
            labels = [label.get_text() for label in ax.get_xticklabels()]
            new_labels = [label if i % 2 == 0 else '' for i, label in enumerate(labels)]
            ax.set_xticklabels(new_labels)
    plt.show()
    
graph_y_time(df[1000:3000], "Imported Orders")
# plot_y_analysis(df, "Imported Orders")


In [ ]:
# 4. Create differenced and scaled y_detrended column
df['y_detrended'] = df['y'].diff().fillna(0)

def graph_y_detrended(df: pd.DataFrame, name: str, feature : str = 'y_detrended'):
    """
    Plot the y_detrended column over time
    
    Parameters:
    df: DataFrame with 'y_detrended' column
    name: Name for the plot title
    """
    plt.figure(figsize=(10, 4))
    plt.plot(df.index, df[feature], label=f'{name} {feature}', color='purple')
    plt.xlabel('Timestamp')
    plt.ylabel(feature)
    plt.title(f'{name}: Differenced and Scaled Y {feature} Over Time')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

graph_y_detrended(df, "Imported Orders")

def plot_acf_pacf(ds, name: str, lag = 200, do_pacf = True):
    """
    Plot ACF and PACF for a given column
    
    Parameters:
    ds: dataseries
    name: Name for the plot title
    """
    # Plot ACF
    fig, ax = plt.subplots(figsize=(12, 6))
    plot_acf(ds, lags=lag, ax=ax)
    ax.set_title(f'Autocorrelation Function (ACF): {name}', fontsize=14)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    if not do_pacf:
        return

    # Plot PACF
    fig, ax = plt.subplots(figsize=(12, 6))
    plot_pacf(ds, lags=lag, ax=ax)
    ax.set_title(f'Partial Autocorrelation Function (PACF): {name}', fontsize=14)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


def plot_periodogram_welch(df: pd.DataFrame, name: str, feature = 'y_detrended'):
    """
    Plot periodogram and Welch's PSD for y_detrended column
    
    Parameters:
    df: DataFrame with 'y_detrended' column
    name: Name for the plot title
    """
    
    # Create subplots
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))
    fig.suptitle(f'Frequency Domain Analysis: {name} {feature}', fontsize=16)
    
    # Periodogram
    frequencies, psd = periodogram(df[feature])
    mask = frequencies > 0
    axes[0].semilogy(frequencies[mask], psd[mask])
    axes[0].set_xlabel('Frequency')
    axes[0].set_ylabel('Power Spectral Density')
    axes[0].set_title('Periodogram')
    axes[0].grid(True, alpha=0.3)
    
    # Welch's PSD
    frequencies_welch, psd_welch = welch(df[feature], fs=1, nperseg=256)
    mask_welch = frequencies_welch > 0
    axes[1].semilogy(frequencies_welch[mask_welch], psd_welch[mask_welch])
    axes[1].set_xlabel('Frequency')
    axes[1].set_ylabel('Power Spectral Density')
    axes[1].set_title("Welch's PSD")
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

    return frequencies_welch, psd_welch

plot_acf_pacf(df['y_detrended'], "Imported Orders", 200)
frequencies, psd = plot_periodogram_welch(df, "Imported Orders")
print(len(df))

peaks, _ = find_peaks(psd, height=np.median(psd)*1)

peak_freqs = frequencies[peaks]
peak_periods = 1 / peak_freqs
print("Significant Periods (in time units):", peak_periods)




In [ ]:

# absolute outliers
filter = HampelFilter(window_length=50, n_sigma = 5.0, return_bool=True)
train_mask = filter.fit_transform(df['y_detrended'])

df.loc[train_mask, 'y'], df.loc[train_mask, 'y_detrended'] = None, None
df.loc[:, 'y'] = (df['y'].interpolate(method='linear', limit_direction='both'))
df.loc[:, 'y_detrended'] = (df['y_detrended'].interpolate(method='linear', limit_direction='both'))


# Apply log transformation to 'y' column in train_df
df['y_log'] = np.log(df['y'])
df['y_detrended'] = df['y_log'].diff().fillna(0)
df['y_detrended_nolog'] = df['y'].diff().fillna(0)

train_df = df[:SPLIT]
test_df = df[SPLIT:]


In [ ]:


def graph_mstl(df: pd.DataFrame, name: str, seasonal_lengths: list, graph = True, feature = 'y_log'):
    """
    Plot Multiple Seasonal-Trend decomposition using LOESS (MSTL)
    
    Parameters:
    df: DataFrame with 'y' column
    name: Name for the plot title
    seasonal_lengths: List of seasonal period lengths (e.g., [24, 168] for daily and weekly patterns)
    """
    from statsmodels.tsa.seasonal import MSTL
    
    # Perform MSTL decomposition
    mstl = MSTL(df[feature], periods=seasonal_lengths, stl_kwargs={'robust': False})
    result = mstl.fit()

    if not graph:
        return result
    
    # Create subplots: original + trend + multiple seasonals + residual
    n_plots = 3 + len(seasonal_lengths)  # original, trend, seasonals, residual
    fig, axes = plt.subplots(n_plots, 1, figsize=(12, 3 * n_plots))
    fig.suptitle(f'MSTL Decomposition: {name} (Periods: {seasonal_lengths})', fontsize=16)
    
    # Plot original series
    axes[0].plot(df.index, df[feature], label='Original', color='black')
    axes[0].set_ylabel('Original')
    axes[0].legend(loc='upper left')
    axes[0].grid(True, alpha=0.3)
    
    # Plot trend
    axes[1].plot(df.index, result.trend, label='Trend', color='blue')
    axes[1].set_ylabel('Trend')
    axes[1].legend(loc='upper left')
    axes[1].grid(True, alpha=0.3)
    
    # Plot each seasonal component
    colors = ['green', 'orange', 'purple', 'red', 'brown', 'pink']
    for i, period in enumerate(seasonal_lengths):
        seasonal = result.seasonal
        axes[2 + i].plot(df.index, seasonal, label=f'Seasonal (period={period})', color=colors[i % len(colors)])
        axes[2 + i].set_ylabel(f'Seasonal_{period}')
        axes[2 + i].legend(loc='upper left')
        axes[2 + i].grid(True, alpha=0.3)
    
    # Plot residual
    axes[-1].plot(df.index, result.resid, label='Residual', color='red', alpha=0.7)
    axes[-1].set_ylabel('Residual')
    axes[-1].set_xlabel('Time (30 mins)')
    axes[-1].legend(loc='upper left')
    axes[-1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return result

# possible seasonal lengths: 24, 772, 1460, 4383 

# Generate all possible non-empty subsets
train_resid = graph_mstl(train_df, "Imported Orders", [48, 336], False).resid.to_frame().rename(columns={'resid': 'y_detrended'})
test_resid = graph_mstl(test_df, "Imported Orders", [48, 336], False).resid.to_frame().rename(columns={'resid': 'y_detrended'})
plot_acf_pacf(train_resid, "Imported Orders", 200, True)



In [ ]:
# data looks clean - now save
# note - df clean removed outliers and entries < 400 because they were during abnormal holiday period
train_df.loc[:, 'resid'] = train_resid['y_detrended']
test_df.loc[:, 'resid'] = test_resid['y_detrended']

# Create holiday mask for Black Friday weekend (4th Thursday of Nov 5:30 AM to following Monday 11:30 PM)
train_df['ds_datetime'] = pd.to_datetime(train_df['ds'])
test_df['ds_datetime'] = pd.to_datetime(test_df['ds'])

# Initialize holiday column
train_df['holiday'] = 0
test_df['holiday'] = 0

# Find Black Friday periods for each year in the dataset
for year in train_df['ds_datetime'].dt.year.unique():
    # Find 4th Thursday of November
    nov_first = pd.Timestamp(f'{year}-11-01')
    # Find first Thursday
    days_until_thursday = (3 - nov_first.dayofweek) % 7
    first_thursday = nov_first + pd.Timedelta(days=days_until_thursday)
    # 4th Thursday is 3 weeks later
    fourth_thursday = first_thursday + pd.Timedelta(weeks=3)
    
    # Black Friday weekend: Thursday 5:30 AM to Monday 11:30 PM
    start_time = fourth_thursday + pd.Timedelta(hours=5, minutes=30)
    end_time = fourth_thursday + pd.Timedelta(days=4, hours=23, minutes=30)
    
    # Mark holiday periods
    train_df.loc[(train_df['ds_datetime'] >= start_time) & (train_df['ds_datetime'] <= end_time), 'holiday'] = 1

for year in test_df['ds_datetime'].dt.year.unique():
    nov_first = pd.Timestamp(f'{year}-11-01')
    days_until_thursday = (3 - nov_first.dayofweek) % 7
    first_thursday = nov_first + pd.Timedelta(days=days_until_thursday)
    fourth_thursday = first_thursday + pd.Timedelta(weeks=3)
    
    start_time = fourth_thursday + pd.Timedelta(hours=5, minutes=30)
    end_time = fourth_thursday + pd.Timedelta(days=4, hours=23, minutes=30)
    
    test_df.loc[(test_df['ds_datetime'] >= start_time) & (test_df['ds_datetime'] <= end_time), 'holiday'] = 1

# Drop temporary datetime column
train_df = train_df.drop('ds_datetime', axis=1)
test_df = test_df.drop('ds_datetime', axis=1)

if 'unique_id' in train_df.columns:
    train_df = train_df.drop('unique_id', axis=1)
    test_df = test_df.drop('unique_id', axis=1)


test_df.insert(0, 'unique_id', 'OrderImport')
train_df.insert(0, 'unique_id', 'OrderImport')


train_df.to_csv('./dataset/cleaned/auth_order_train.csv', index=False)
test_df.to_csv('./dataset/cleaned/auth_order_test.csv', index=False)